In [1]:
from urllib.request import urlopen, urlretrieve
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import time
import requests 
from fake_headers import Headers 



                



## Балтздрав

In [2]:
baltzdrav_services_list = {}  # сюда запишем все страницы для поиска

resp = urlopen('http://baltzdrav.ru/services') # скачиваем файл
html = resp.read().decode('utf8') # считываем содержимое
soup = BeautifulSoup(html, 'html.parser') # делаем суп

for link in soup.find_all('li'):
    if link.has_attr('class') and "uk-none" in link.get('class'):
        for a in link.find_all('a', href=True):
            baltzdrav_services_list.setdefault(a.text, a.get('href'))
            
adress = 'http://baltzdrav.ru'
baltzdrav = {'code' : [],
            'description' : [],
            'price' : [],
            'department' : []}

for key, val in baltzdrav_services_list.items():

    resp = urlopen(adress + val) # скачиваем файл
    html = resp.read().decode('utf8') # считываем содержимое
    soup = BeautifulSoup(html, 'html.parser') # делаем суп
    cnt = 0
    for link in soup.find_all('td'):
        if link.has_attr('class'):
            attribute = link.get('class')
            if re.search(r'xl\d{2}', str(attribute)) and re.search(r'^\d{5}$', link.text):
                baltzdrav['code'].append(link.text)
                baltzdrav['description'].append(link.find_next('td').text)
                baltzdrav['price'].append(link.find_next('td').find_next('td').text)
                baltzdrav['department'].append(key) 


    
data_baltzdrav = pd.DataFrame.from_dict(baltzdrav)
data_baltzdrav.columns = ['Код', 'Описание услуги', 'Цена в рублях', 'Отдел']
data_baltzdrav['Цена в рублях'] = data_baltzdrav['Цена в рублях'].apply(lambda x: float(x.replace(' ', '')))
data_baltzdrav.to_excel(r'C:\Users\AmanovRA\Anaconda3\Work-work\Parsings\Балтздрав\Baltzdrav_' + str(datetime.date(datetime.today())) + '.xlsx')    
        

## Скандинавия

In [3]:
avaclinic = {'name' : [],
            'price' : []}



resp = urlopen('https://avaclinic.ru/prices/') 
html = resp.read().decode('utf8') 
soup = BeautifulSoup(html, 'html.parser') 
lastpage = 0 
pattern = r'/prices/\?PAGEN_1=(\d+)'

for link in soup.find_all('a'):
    if link.has_attr('href') and '/prices/?PAGEN_1=' in link.get('href'):
        if int(re.search(pattern, link.get('href')).groups(1)[0]) > lastpage:
            lastpage = int(re.search(pattern, link.get('href')).groups(1)[0]) # нашли последнюю страницу в прайсах из ul pagination чтобы знать до какой парсить

            
adress = 'https://avaclinic.ru/prices/?PAGEN_1='
cnt = 1
          

while cnt <= lastpage:
    resp = urlopen(adress + str(cnt)) 
    html = resp.read().decode('utf8') 
    soup = BeautifulSoup(html, 'html.parser') 

    for link in soup.find_all('p'):
        if link.has_attr('class') and link.get('class')[0] == 'name':
            avaclinic['name'].append(link.text)
        if link.has_attr('class') and link.get('class')[0] == "price":
            avaclinic['price'].append(float(''.join([i for i in link.text if i.isdigit()])))
            
    cnt += 1           
data_avaclinic = pd.DataFrame.from_dict(avaclinic)
data_avaclinic.columns = ['Описание услуги', 'Цена в рублях']
data_avaclinic.to_excel(r'C:\Users\AmanovRA\Anaconda3\Work-work\Parsings\Скандинавия\Scandinav_' + str(datetime.date(datetime.today())) + '.xlsx')        

## СМ-Клиника

In [4]:
smclinic = {'name' : [],
            'price' : []}

resp = urlopen('https://www.smclinic-spb.ru/uslugi/tseny-i-stoimost-uslug')
html = resp.read().decode('utf8') 
soup = BeautifulSoup(html, 'html.parser')

for link in soup.find_all('td'):
    if link.has_attr('class') and link.get('class')[0] == 'price' and len(link.get('class')) == 1:
        smclinic['name'].append(link.find_previous('td').text)
        smclinic['price'].append(float(''.join([i for i in link.text if i.isdigit()])))

data_smclinic = pd.DataFrame.from_dict(smclinic)
data_smclinic.columns = ['Описание услуги', 'Цена в рублях']
data_smclinic.to_excel(r'C:\Users\AmanovRA\Anaconda3\Work-work\Parsings\СМ_Клиник\SM_clinic_' + str(datetime.date(datetime.today())) + '.xlsx')        

## Немецкая семейная клиника

In [5]:
directions = {'departments' : [],
             'service' : [],
             'price' : []}
pattern1 = r'(.+%)(.+)'  # убрать цифры с процентами
pattern2 = r'[^\d ]'
home_pages = ['https://german.clinic', 'https://german.dental']

for index_home_page, home_page in enumerate(home_pages):
    if index_home_page == 0:
        resp = urlopen(home_page)
        html = resp.read().decode('utf8') 
        soup = BeautifulSoup(html, 'html.parser')
        for ul in soup.find_all('ul', "services__list"):
            for li in ul.find_all('li'):
                link = li.find('a')
                if '//' not in link.get('href'):
                    age = 'children' if 'children' in link.get('href') else 'adult' if 'adult' in link.get('href') else ''
                    try: # некоторые ссылки битые на услуги
                        resp = urlopen(home_page + link.get('href'))
                    except:
                        continue
                    html = resp.read().decode('utf8') 
                    soup = BeautifulSoup(html, 'html.parser')
                    if soup.find('table', "textpage__table"):
                        table = soup.find('table', "textpage__table")
                        for tr in table.find_all('tr'):
                            if tr.find('td'):
                                if (len(tr.find_all('td'))) == 1:
                                    continue
                                td = tr.find('td')
                                td_price = td.find_next('td').text
                                if td_price == '':
                                    continue
                                directions['service'].append(td.text)     
                                td_price = re.sub(pattern1, r'\2', td_price)
                                td_price = re.sub(pattern2, '', td_price).strip(' ').split()
                                tempprices = []
                                for i, j in enumerate(td_price[::2]):
                                    if len(j) in (1, 2):
                                        tempprices.append(td_price[i * 2] + td_price[i * 2 + 1])
                                    elif len(td_price) == 1:
                                        tempprices.append(j)
                                    elif len(j) >= 4:
                                        tempprices.append(j)               
                                directions['price'].append(tempprices)
                                directions['departments'].append(link.text + ' ' + age)
    else:
        resp = urlopen(home_page)
        html = resp.read().decode('utf8') 
        soup = BeautifulSoup(html, 'html.parser')
        for li in soup.find_all('li', "topmenu__item"):
            if "data-rel" in li.find_next('a').attrs:
                ul = li.find_next('a', {"data-rel" : "#service"}).find_next('ul')
                for service_li in ul.find_all('li'):
                    link = service_li.find_next('a')
                    age = 'children' if 'children' in link.get('href') else 'adult' if 'adult' in link.get('href') else ''
                    try: # некоторые ссылки битые на услуги
                        resp = urlopen(home_page + link.get('href'))
                    except:
                        continue
                    html = resp.read().decode('utf8') 
                    soup = BeautifulSoup(html, 'html.parser')
                    if soup.find('table', "textpage__table"):
                        table = soup.find('table', "textpage__table")
                        for tr in table.find_all('tr'):
                            if tr.find('td'):
                                if (len(tr.find_all('td'))) == 1:
                                    continue
                                td = tr.find('td')
                                td_price = td.find_next('td').text
                                if td_price == '':
                                    continue
                                directions['service'].append(td.text)     
                                td_price = re.sub(pattern1, r'\2', td_price)
                                td_price = re.sub(pattern2, '', td_price).strip(' ').split()
                                tempprices = []
                                for i, j in enumerate(td_price[::2]):
                                    if len(j) in (1, 2):
                                        tempprices.append(td_price[i * 2] + td_price[i * 2 + 1])
                                    elif len(td_price) == 1:
                                        tempprices.append(j)
                                    elif len(j) >= 4:
                                        tempprices.append(j)               
                                directions['price'].append(tempprices)
                                directions['departments'].append(link.text + ' ' + age)        
                        

data = pd.DataFrame(directions)

def join_list(lst):
    if type(lst) == list and len(lst) > 1:
        return '; '.join([i for i in lst])
    elif len(lst) == 1 and lst[0].isnumeric():
        return int(lst[0])
    
data.price = data.price.apply(join_list)


data.to_excel(r'C:\Users\AmanovRA\Anaconda3\Work-work\Parsings\Немецкая_семейная\German_clinic_' + str(datetime.date(datetime.today())) + '.xlsx')